<a href="https://colab.research.google.com/github/samikshaa0604/FUTURE_ML_01/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# Cell 1: Install Prophet (if not already installed)
!pip install prophet


In [16]:
# Cell 2: Import libraries
import pandas as pd
from prophet import Prophet
from google.colab import files

In [17]:
# Cell 3: Load dataset (you will upload and give path here)
# Example: df = pd.read_csv('/content/SampleSuperstore.csv')
df = pd.read_csv('/content/drive/MyDrive/future interns datasets/Sample - Superstore.csv', encoding='latin1')


# Convert dates
df['Order Date'] = pd.to_datetime(df['Order Date'])


In [18]:
# Cell 4: Create year and month features
df['year'] = df['Order Date'].dt.year
df['month'] = df['Order Date'].dt.month


In [19]:
# Cell 5: Aggregate data at required level
agg_df = df.groupby(
    ['year','month','Region','Category','Sub-Category','Product Name'],
    as_index=False
).agg({
    'Sales':'sum',
    'Profit':'sum',
    'Quantity':'sum'
})

agg_df.rename(columns={
    'Region':'region',
    'Category':'category',
    'Sub-Category':'sub_category',
    'Product Name':'product_name',
    'Sales':'total_sales',
    'Profit':'total_profit',
    'Quantity':'quantity'
}, inplace=True)


In [20]:
# Cell 6: Prepare data for forecasting (overall sales by month)
monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({'Sales':'sum'}).reset_index()
monthly_sales.rename(columns={'Order Date':'ds','Sales':'y'}, inplace=True)


/tmp/ipython-input-2369101067.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({'Sales':'sum'}).reset_index()


In [21]:
# Cell 7: Fit Prophet model
model = Prophet(interval_width=0.95)
model.fit(monthly_sales)

future = model.make_future_dataframe(periods=6, freq='M')  # 6 months forecast
forecast = model.predict(future)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6ut36gpt/413ws_ne.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp6ut36gpt/imlj5ieo.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84073', 'data', 'file=/tmp/tmp6ut36gpt/413ws_ne.json', 'init=/tmp/tmp6ut36gpt/imlj5ieo.json', 'output', 'file=/tmp/tmp6ut36gpt/prophet_modelbs8p5ipl/prophet_model-20251021060639.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
06:06:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:06:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.12/dist-packages

In [22]:
# Cell 8: Merge forecast with agg_df
# Extract year and month from forecast
forecast['year'] = forecast['ds'].dt.year
forecast['month'] = forecast['ds'].dt.month

forecast = forecast[['year','month','yhat','yhat_lower','yhat_upper']]
forecast.rename(columns={
    'yhat':'forecast_sales',
    'yhat_lower':'forecast_lower_ci',
    'yhat_upper':'forecast_upper_ci'
}, inplace=True)

# Add actual sales
actual = monthly_sales.copy()
actual['year'] = actual['ds'].dt.year
actual['month'] = actual['ds'].dt.month
actual = actual[['year','month','y']]
actual.rename(columns={'y':'actual_sales'}, inplace=True)

# Merge forecast + actual into agg_df
final_df = agg_df.merge(actual, on=['year','month'], how='left')
final_df = final_df.merge(forecast, on=['year','month'], how='left')


In [23]:
# Cell 9: Save and download final CSV
final_df.to_csv('final_output.csv', index=False)
files.download('final_output.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>